In [1]:
import random
from IPython.display import clear_output
from cryptography.fernet import Fernet
# Загружаем ключ 'crypto.key' из текущего каталога
def Load_key():
    return open('crypto.key', 'rb').read()
# Загружаем файлы сохранения из текущего каталога, если они в нем есть, дешифруем один из них и собираем карты игрока и игры
def Load():
    try:
        file = open('save.txt')
    except IOError as e:
        print(u'файла сохранения нет,начнем сначала')
    else:
        with file:
            suit = Fernet(Load_key())
            cnt = 0
            file = open('save.txt','rb')
            decoded_text = suit.decrypt(file.read())
            arr = str(decoded_text.decode("utf-8")).replace('[','').replace(' ','').replace("'",'').split('],')
            arr[-1] = arr[-1].replace(']','')
            for i in range(len(arr)):
                arr[i] = arr[i].split(',')
            for i in range(len(arr)):
                for j in range(len(arr[i])):
                    if(arr[i][j]!='X'):
                        arr[i][j] = int(arr[i][j])
                    else:
                        cnt+=1
    file = open('save_player.txt')
    player_arr = str(file.read()).replace('[','').replace(' ','').replace("'",'').split('],')
    player_arr[-1] = player_arr[-1].replace(']','')
    for i in range(len(player_arr)):
                player_arr[i] = player_arr[i].split(',')
    return player_arr,arr,len(arr[1]),cnt
# Сохраняем ключ шифрования в отдельный файл в текущем каталоге
def Write_key():
    key = Fernet.generate_key()
    with open('crypto.key', 'wb') as key_file:
        key_file.write(key)
    return key
# Сохраняем карты игрока и игры в текущий каталог и шифруем вторую
def Save(arr,player_arr):
    key = Write_key()
    suit = Fernet(key)
    arr_save = bytes(str(arr),'utf-8')
    arr_player_save = str(player_arr)
    f = open('save.txt', 'wb' )
    f.write(suit.encrypt(arr_save))
    f.close()
    f = open('save_player.txt', 'w' )
    for item in arr_player_save:
        for i in item:
            f.write(i)
    f.close()
    return
# в случае, если нет сохранений, генерим карты игрока и игры и правильно расставляем цифры
def Create_map(load):
    if load==1:
        return Load()
    #создаем шаблон поля размера n*n
    n = int(input("Введите размер поля:"))
    k = int(input("Введите кол-во бомб:"))
    arr = [[0] * n for i in range(n)]
    player_arr = [['-'] * n for i in range(n)]
    for i in range(k):
        bomb_x = random.randint(0,n-1)
        bomb_y = random.randint(0,n-1)
        # проверяем нет ли уже бомбы на наших координатах
        if(arr[bomb_y][bomb_x] == 'X'):
            while(arr[bomb_y][bomb_x] == 'X'):
                bomb_x = random.randint(0,n-1)
                bomb_y = random.randint(0,n-1)
        arr[bomb_y][bomb_x] = 'X'
        # заполняем элементы вокруг бомб числами
        # убираем возможность выйти за пределы индексации двумерного списка и проверяем нет ли там, где нам нужно увеличить число, 'X'
        if (bomb_x >= 1 and bomb_x <= n-1) and (bomb_y >= 1 and bomb_y <= n-1) and (arr[bomb_y-1][bomb_x-1]!='X'): #лево верх
            arr[bomb_y-1][bomb_x-1] += 1 
        if (bomb_x >= 0 and bomb_x <= n-1) and (bomb_y >= 1 and bomb_y <= n-1) and (arr[bomb_y-1][bomb_x]!='X'): #центр верх
            arr[bomb_y-1][bomb_x] += 1 
        if (bomb_x >= 0 and bomb_x <= n-2) and (bomb_y >= 1 and bomb_y <= n-1) and (arr[bomb_y-1][bomb_x+1]!='X'): #право верх
            arr[bomb_y-1][bomb_x+1] += 1 
        if (bomb_x >=1 and bomb_x <= n-1) and (bomb_y >= 0 and bomb_y <= n-1) and (arr[bomb_y][bomb_x-1]!='X'): # лево центр
            arr[bomb_y][bomb_x-1] += 1 
        if (bomb_x >=0 and bomb_x <= n-2) and (bomb_y >= 0 and bomb_y <= n-1) and (arr[bomb_y][bomb_x+1]!='X'): # право центр
            arr[bomb_y][bomb_x+1] += 1 
        if (bomb_y >= 1 and bomb_x <= n-1) and (bomb_y >= 0 and bomb_y <= n-2) and (arr[bomb_y+1][bomb_x-1]!='X'): # лево низ
            arr[bomb_y+1][bomb_x-1] += 1 
        if (bomb_x >= 0 and bomb_x <= n-1) and (bomb_y >= 0 and bomb_y <= n-2) and (arr[bomb_y+1][bomb_x]!='X'): # центр низ
            arr[bomb_y+1][bomb_x] += 1 
        if (bomb_x >=0 and bomb_x <= n-2) and (bomb_y >= 0 and bomb_y <= n-2) and (arr[bomb_y+1][bomb_x+1]!='X'): #центр право
            arr[bomb_y+1][bomb_x+1] += 1 
    return(player_arr,arr,n,k)
#более-менее приемлемое отображение карт на экране
def show_arr(arr):
    for row in arr:
        print(" ".join(str(cell) for cell in row))
        print("")
#сам алгоритм игры
def Game():
    load = int(input('Загрузить игру(введите 1) или начать новую(введите 0)?'))            
    player_arr,arr,n,k = Create_map(load)
    score = 0
    flag_count = k
    while True:
        clear_output()
        # условие победы
        if(score==(n*n-k)):
            print("Ты выиграл!")
            break
        show_arr(player_arr)
        print("Кол-во оставшихся флагов: " + str(flag_count))
        print("Введите координанты точки и действие через запятую [X,Y,Action],save если вы хотите сохраниться или exit:")
        inp = str(input())
        #считываем input и выполняем нужное пользователю дейсвтие
        if(inp =='save'):
            Save(arr,player_arr)
            continue
        if(inp == 'exit'):
            Save(arr,player_arr)
            print('Спасибо за игру!')
            break
        inp_arr = inp.replace(" ","").split(',') 
        if(len(inp_arr) == 3):
            x = int(inp_arr[0]) - 1 
            y = int(inp_arr[1]) - 1
            action = inp_arr[2].lower()
        else:
            x = int(inp_arr[0]) - 1 
            y = int(inp_arr[1]) - 1
            action = 'nothing'
        if(action=='flag'):
            if(flag_count != 0 or player_arr[y][x]!= 'f'):
                player_arr[y][x] = 'f'
                flag_count -=1
                continue
            else:
                continue
        # условие проигрыша
        if (arr[y][x] == 'X'):
            print("Game Over!")
            show_arr(arr)
            break
        else:
            player_arr[y][x] = arr[y][x]
            score+=1
    return()
Game()

f - - -

- 1 f -

- - - -

- - - -

Кол-во оставшихся флагов: 3
Введите координанты точки и действие через запятую [X,Y,Action],save если вы хотите сохраниться или exit:
exit


()

In [57]:
from cryptography.fernet import Fernet
key = Fernet.generate_key() #this is your "password"
with open('crypto.key', 'wb') as key_file:
        key_file.write(key)
key = load_key()
cipher_suite = Fernet(key)
encoded_text = cipher_suite.encrypt(b"Hello stackoverflow!")
decoded_text = cipher_suite.decrypt(encoded_text)

In [58]:
decoded_text

b'Hello stackoverflow!'

In [56]:
def load_key():
# Загружаем ключ 'crypto.key' из текущего каталога
    return open('crypto.key', 'rb').read()

In [39]:
from cryptography.fernet import Fernet
key = Fernet.generate_key() #this is your "password"
cipher_suite = Fernet(key)
encoded_text = cipher_suite.encrypt(b"Hello stackoverflow!")
decoded_text = cipher_suite.decrypt(encoded_text)

In [43]:
decoded_text.decode("utf-8")

'[[0, 0], [0, 0]]'